In [1]:
import pandas as pd
import pandas as pd
import torch
import pickle
import os

import nlpsig
from nlpsig.deepsignet import DeepSigNet, StackedDeepSigNet
from nlpsig.focal_loss import FocalLoss

In [81]:
int(False)

0

In [82]:
def concatenate_data(data_folder_path):
    print(f"looking in {data_folder_path} directory...")
    manifesto_dfs = []
    for filename in os.listdir(data_folder_path):
        print(f"- reading {filename}...")
        # parse filename for metadata
        filename_split = filename.split("_")
        party_id = int(filename_split[0])
        year = int(filename_split[1][0:4])
        month = int(filename_split[1][4:6])
        doc_id = f"{party_id}_{year}"
        # read dataframe and add metadata
        df = pd.read_csv(f"data/{filename}")[["text", "cmp_code"]]
        df = df[df["cmp_code"]!="H"].dropna().reset_index(drop=True)
        df["topic"] = [int(str(code)[0]) for code in df["cmp_code"]]
        df["switched_topic"] = [1] + [int(df["topic"].iloc[i]!=df["topic"].iloc[i-1])
                                      for i in range(1, len(df))]
        df["party_id"] = party_id
        df["doc_id"] = f"{party_id}_{year}"
        df["datetime"] = pd.Timestamp(f"{year}-{month}")
        manifesto_dfs.append(df)
    return pd.concat(manifesto_dfs).reset_index(drop=True)

In [83]:
manifesto_df = concatenate_data("data/")

looking in data/ directory...
- reading 51902_201706.csv...
- reading 51902_201505.csv...
- reading 51320_201912.csv...
- reading 51620_201706.csv...
- reading 51620_201505.csv...
- reading 51421_201706.csv...
- reading 51421_201505.csv...
- reading 51421_201912.csv...
- reading 51902_201912.csv...
- reading 51320_201706.csv...
- reading 51620_201912.csv...
- reading 51320_201505.csv...


['text_0',
 'text_1',
 'text_2',
 'text_3',
 'text_4',
 'text_5',
 'text_6',
 'text_7',
 'text_8',
 'text_9',
 'text_10',
 'text_11',
 'text_12',
 'text_13',
 'text_14',
 'text_15',
 'text_16',
 'text_17',
 'text_18',
 'text_19']

In [236]:
import random
random.seed(2022)
n_entries = 1000
list_datetimes = pd.to_datetime([
            "2015-01-01 00:00:00",
            "2015-01-01 00:12:00",
            "2015-01-02 00:00:00",
            "2015-01-02 00:12:00",
            "2015-01-03 00:00:00",
            "2015-01-03 00:12:00",
            "2015-01-04 00:00:00",
            "2015-01-04 00:12:00",
            "2015-01-05 00:00:00",
            "2015-01-05 00:12:00",
            "2015-01-06 00:00:00",
            "2015-01-06 00:12:00",
        ])
test_df = pd.DataFrame({"text": [f"text_{i}" for i in range(n_entries)], 
                        "id_col": [random.randint(1,10) for i in range(n_entries)],
                        "label_col": [random.randint(0, 4) for i in range(n_entries)],
                        "datetime": [random.choice(list_datetimes) for i in range(n_entries)]})
test_df

,text,id_col,label_col,datetime
0,text_0,9,2,2015-01-04 00:12:00
1,text_1,5,2,2015-01-06 00:00:00
2,text_2,8,3,2015-01-05 00:00:00
3,text_3,9,1,2015-01-01 00:12:00
4,text_4,5,3,2015-01-06 00:00:00
...,...,...,...,...
995,text_995,9,1,2015-01-03 00:00:00
996,text_996,6,3,2015-01-01 00:12:00
997,text_997,4,2,2015-01-05 00:00:00
998,text_998,10,0,2015-01-02 00:12:00


In [237]:
emb = np.random.rand(1000, 300)
reduced_emb = np.random.rand(1000, 10)

In [234]:
test = PrepareData(test_df, , id_column="id_col")

[INFO] Adding time feature columns into dataframe in .df


In [235]:
test.df

,text,id_col,label_col,datetime,e1,e2,e3,e4,e5,e6,...,e294,e295,e296,e297,e298,e299,e300,time_encoding,time_diff,timeline_index
0,text_80,1,1,2015-01-01 00:00:00,0.978538,0.964930,0.967513,0.348821,0.679717,0.533678,...,0.296889,0.213993,0.899575,0.207499,0.729210,0.799419,0.677181,2015.000000,0,1
1,text_125,1,3,2015-01-01 00:00:00,0.244572,0.070819,0.421841,0.317372,0.125129,0.253760,...,0.843843,0.550103,0.820107,0.823241,0.057735,0.139715,0.931037,2015.000000,0,2
2,text_152,1,3,2015-01-01 00:00:00,0.236084,0.573905,0.219066,0.555015,0.560313,0.154235,...,0.225230,0.221659,0.915657,0.692285,0.137242,0.770958,0.167274,2015.000000,0,3
3,text_154,1,3,2015-01-01 00:00:00,0.137550,0.260595,0.285909,0.343901,0.215406,0.407291,...,0.442239,0.026531,0.112764,0.007675,0.398971,0.322033,0.114601,2015.000000,0,4
4,text_179,1,3,2015-01-01 00:00:00,0.733311,0.457557,0.655394,0.693147,0.883235,0.810205,...,0.248873,0.125703,0.382511,0.600995,0.543653,0.758271,0.913017,2015.000000,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,text_753,10,0,2015-01-06 00:12:00,0.290857,0.763638,0.830212,0.077629,0.939260,0.113435,...,0.821199,0.268650,0.023907,0.321583,0.239362,0.833275,0.172613,2015.013721,0,92
996,text_761,10,3,2015-01-06 00:12:00,0.657563,0.572087,0.655395,0.522517,0.201079,0.686710,...,0.031143,0.687310,0.913161,0.005045,0.933846,0.409109,0.735475,2015.013721,0,93
997,text_793,10,0,2015-01-06 00:12:00,0.499214,0.795711,0.454675,0.520025,0.511093,0.486659,...,0.036459,0.849619,0.470995,0.918695,0.313807,0.338792,0.328943,2015.013721,0,94
998,text_888,10,1,2015-01-06 00:12:00,0.215515,0.505435,0.933015,0.326396,0.629620,0.256354,...,0.454103,0.352716,0.783519,0.943629,0.954366,0.393293,0.193424,2015.013721,0,95


In [227]:
list_datetimes[1]

Timestamp('2015-01-01 00:12:00')

In [198]:
manifesto_df.head(10)

,text,cmp_code,topic,switched_topic,party_id,doc_id,datetime
0,SNP MPs have used their influence to deliver p...,305.1,3,1,51902,51902_2017,2017-06-01
1,Here’s just some of what a strong team of SNP ...,305.1,3,0,51902,51902_2017,2017-06-01
2,When the Scotland Bill was going through Westm...,301,3,0,51902,51902_2017,2017-06-01
3,"And it was SNP MPs, working with the Scottish ...",301,3,0,51902,51902_2017,2017-06-01
4,The SNP secured a deal that ensures Scotland w...,301,3,0,51902,51902_2017,2017-06-01
5,SNP MPs have consistently opposed Tory austerity.,504,5,1,51902,51902_2017,2017-06-01
6,Our MPs have been instrumental in forcing UK g...,504,5,0,51902,51902_2017,2017-06-01
7,Alison Thewliss has been at the forefront of t...,504,5,0,51902,51902_2017,2017-06-01
8,and force women to prove they have been raped ...,503,5,0,51902,51902_2017,2017-06-01
9,SNP MPs have worked with Women Against State P...,503,5,0,51902,51902_2017,2017-06-01


In [85]:
manifesto_df["switched_topic"].value_counts()

0    10746
1     4864
Name: switched_topic, dtype: int64

In [86]:
manifesto_df["topic"].value_counts()

5    5300
4    3872
6    1547
1    1504
2    1235
3    1101
7    1024
0      27
Name: topic, dtype: int64

In [87]:
manifesto_df["party_id"].value_counts()

51421    4515
51620    4307
51320    4039
51902    2749
Name: party_id, dtype: int64

In [88]:
manifesto_df["doc_id"].value_counts()

51421_2015    1917
51320_2019    1702
51620_2015    1588
51620_2017    1496
51421_2019    1467
51320_2017    1328
51620_2019    1223
51421_2017    1131
51902_2019    1071
51320_2015    1009
51902_2015     892
51902_2017     786
Name: doc_id, dtype: int64

## Model specifics

Nested dictionary for models specifications.

This includes models for encoding text, path signature and etc.

In [89]:
model_specifics = {
    "encoder_args": {
        "col_name_text": "text", # column corresponding to the sentences
        "model_name": "all-MiniLM-L6-v2", #options: all-mpnet-base-v2, all-distilroberta-v1, all-MiniLM-L12-v2
        "model_args": {
            "batch_size": 64,
            "show_progress_bar": True,
            "output_value": 'sentence_embedding', 
            "convert_to_numpy": True,
            "convert_to_tensor": False,
            "device": None,
            "normalize_embeddings": False
        }
    },
    "dim_reduction": {
        "method": 'umap', #options: ppapca, ppapcappa, umap
        "n_components": 10, # options: any int number between 1 and embedding dimensions
    },
    "embedding":{
        "global_embedding_tp": 'SBERT', #options: SBERT, BERT_cls , BERT_mean, BERT_max
        "post_embedding_tp": 'sentence', #options: sentence, reduced
        "feature_combination_method": 'attention', #options concatenation, attention 
    },
    "time_injection": {
        "history_tp": 'timestamp', #options: timestamp, None
        "post_tp": 'timestamp', #options: timestamp, timediff, None
    },
    "signature": {
        "dimensions": 3, #options: any int number larger than 1
        "method": 'log', # options: log, sig
        "interval": 1/12
    }
}

## Obtaining SBERT Embeddings

We can use the `TextEncoder` class within `nlpsig` to obtain sentence embeddings from a model. Here, we have defined the encoder arguments in `model_specifics`.

In [90]:
model_specifics["encoder_args"]

{'col_name_text': 'text',
 'model_name': 'all-MiniLM-L6-v2',
 'model_args': {'batch_size': 64,
  'show_progress_bar': True,
  'output_value': 'sentence_embedding',
  'convert_to_numpy': True,
  'convert_to_tensor': False,
  'device': None,
  'normalize_embeddings': False}}

We can pass these into the constructor of the class to initialise our text encoder as follows:

In [91]:
# initialise the Text Encoder 
text_encoder = nlpsig.TextEncoder(manifesto_df, **model_specifics["encoder_args"])

The class has a `.encode_sentence_transformer()` method which first loads in the model (using the `model_name` and `model_args` attributes) and then obtains an embedding for each sentence. These sentence embeddings are then stored in the `embeddings_sentence` attribute of the object.

In [92]:
text_encoder.encode_sentence_transformer()
embeddings_sentence = text_encoder.embeddings_sentence

[INFO] number of sentences to encode: 15610


Batches:   0%|          | 0/244 [00:00<?, ?it/s]

## Dimensionality Reduction with UMAP

Here we specified our choices in `model_specifics` above:

In [93]:
model_specifics["dim_reduction"]

{'method': 'umap', 'n_components': 10}

In [94]:
reduction = nlpsig.DimReduce(**model_specifics["dim_reduction"])
embeddings_reduced = reduction.fit_transform(embeddings_sentence)

  1%|▋                                                                                              | 104/15610 [09:56<24:42:24,  5.74s/it]


In [95]:
print(embeddings_sentence.shape)
print(embeddings_reduced.shape)

(15610, 384)
(15610, 10)


## Data preparation: Time injection and Padding

In [151]:
manifesto_data = PrepareData(manifesto_df,
                                    id_column="doc_id",
                                    labels_column="switched_topic",
                                    embeddings=embeddings_sentence,
                                    embeddings_reduced=embeddings_reduced)

[INFO] Adding time feature columns into dataframe in .df


In [152]:
manifesto_data.df["doc_id"].value_counts()

51421_2015    1917
51320_2019    1702
51620_2015    1588
51620_2017    1496
51421_2019    1467
51320_2017    1328
51620_2019    1223
51421_2017    1131
51902_2019    1071
51320_2015    1009
51902_2015     892
51902_2017     786
Name: doc_id, dtype: int64

In [153]:
manifesto_data.df[manifesto_data.df["doc_id"]=="51902_2017"]

,text,cmp_code,topic,switched_topic,party_id,doc_id,datetime,d1,d2,d3,...,e378,e379,e380,e381,e382,e383,e384,time_encoding,time_diff,timeline_index
13753,SNP MPs have used their influence to deliver p...,305.1,3,1,51902,51902_2017,2017-06-01,7.076446,4.272100,7.765581,...,0.005326,0.037123,0.016147,0.055606,0.008570,0.026209,-0.021132,2017.413699,0,1
13754,Here’s just some of what a strong team of SNP ...,305.1,3,0,51902,51902_2017,2017-06-01,6.945519,4.231819,7.687037,...,-0.022241,-0.003946,0.018906,0.033558,-0.001041,0.002696,0.039106,2017.413699,0,2
13755,When the Scotland Bill was going through Westm...,301,3,0,51902,51902_2017,2017-06-01,7.170236,4.472499,8.550349,...,-0.037495,-0.051820,0.005708,0.042825,0.048136,0.034429,-0.040953,2017.413699,0,3
13756,"And it was SNP MPs, working with the Scottish ...",301,3,0,51902,51902_2017,2017-06-01,7.223161,4.530838,8.441557,...,-0.020897,-0.088878,-0.045942,0.015616,-0.015972,0.036915,0.012497,2017.413699,0,4
13757,The SNP secured a deal that ensures Scotland w...,301,3,0,51902,51902_2017,2017-06-01,7.302962,4.455559,8.645349,...,0.020309,-0.034161,-0.037054,-0.051635,-0.082328,-0.005767,-0.041583,2017.413699,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14534,The disgraceful condition of the housing provi...,201.2,2,0,51902,51902_2017,2017-06-01,5.455401,3.690897,8.006092,...,-0.058051,-0.012283,0.056241,-0.059958,-0.062718,0.068832,0.033046,2017.413699,0,782
14535,The Scottish Government’s work to resettle Syr...,201.2,2,0,51902,51902_2017,2017-06-01,5.445345,4.447828,7.652509,...,0.040820,0.043064,0.039066,0.044445,0.076865,0.004159,0.005174,2017.413699,0,783
14536,We will urge the UK government to work with th...,301,3,1,51902,51902_2017,2017-06-01,5.239152,4.309934,7.645834,...,-0.008096,0.020132,0.048608,0.039107,0.038743,-0.023217,-0.010533,2017.413699,0,784
14537,rather than use private contractors who have p...,413,4,1,51902,51902_2017,2017-06-01,5.336844,2.650033,8.115968,...,0.004935,-0.016151,0.000850,-0.079934,-0.067935,0.046949,0.015082,2017.413699,0,785


In [173]:
history_path = manifesto_data.pad(pad_by="history",
                                  method="k_last",
                                  zero_padding=True,
                                  k=1,
                                  time_feature="timeline_index",
                                  standardise_time_feature=False,
                                  embeddings="dim_reduced")

[INFO] Padding ids and storing in .df_padded and .array_padded attributes


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 15610/15610 [00:20<00:00, 776.79it/s]


In [174]:
history_path.shape

(15610, 1, 13)

In [175]:
manifesto_data.label_column

'switched_topic'

In [178]:
history_path[2]

array([[2, 5.111911773681641, 3.5011675357818604, 7.9254584312438965,
        10.427611351013184, 3.838214874267578, 9.157601356506348,
        6.385989189147949, 4.594970703125, 1.2000325918197632,
        3.1077473163604736, '51320_2015', 0]], dtype=object)

Baselines:
   - just looking at the sentence embeddings (encodes nothing about the history on the post)
       - highlights importance of looking at the sequence
   - averaging history
   - comparing the cosine similarity between previous post and current post to see if switch
   
Test for:
- How many posts do you need to look back?

In [180]:
manifesto_data.df["doc_id"].value_counts()

51421_2015    1917
51320_2019    1702
51620_2015    1588
51620_2017    1496
51421_2019    1467
51320_2017    1328
51620_2019    1223
51421_2017    1131
51902_2019    1071
51320_2015    1009
51902_2015     892
51902_2017     786
Name: doc_id, dtype: int64

In [179]:
manifesto_data.df_padded[manifesto_data.df_padded["doc_id"]=="51320_2015"]

,timeline_index,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,doc_id,switched_topic
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51320_2015,-1
1,1,5.452659,3.602911,8.050925,10.075459,3.911385,9.213820,5.951643,4.275470,1.188564,2.751409,51320_2015,1
2,2,5.111912,3.501168,7.925458,10.427611,3.838215,9.157601,6.385989,4.594971,1.200033,3.107747,51320_2015,0
3,3,5.043742,3.522960,7.468404,9.604556,3.859646,9.537117,6.133909,4.508759,0.930652,2.949600,51320_2015,0
4,4,5.246913,3.351569,7.850750,8.427454,4.078413,9.972346,6.232757,4.770494,1.519472,2.442487,51320_2015,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,1004,4.939366,2.744935,6.556887,9.778514,5.032885,9.680021,8.184111,4.553108,1.828841,2.317884,51320_2015,1
1005,1005,3.629394,2.353125,7.404681,10.038898,5.321763,9.898950,7.223835,4.316514,2.549977,2.290533,51320_2015,1
1006,1006,4.761509,3.163177,7.822935,10.507176,4.688953,8.588090,8.575832,4.712253,1.761940,2.563453,51320_2015,1
1007,1007,4.707180,3.207190,7.543126,10.101868,4.077579,9.360678,6.383281,4.666095,1.278698,3.281799,51320_2015,0


In [183]:
manifesto_data.get_torch_path(include_time_features=False).shape

torch.Size([15610, 1, 10])

In [72]:
history_path[3]

array([[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        '51320_2015', -1],
       [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        '51320_2015', -1],
       [1, 5.452658653259277, 3.6029112339019775, 8.050925254821777,
        10.075458526611328, 3.9113845825195312, 9.213820457458496,
        5.951642990112305, 4.27547025680542, 1.1885640621185303,
        2.751408576965332, '51320_2015', 1],
       [2, 5.111911773681641, 3.5011675357818604, 7.9254584312438965,
        10.427611351013184, 3.838214874267578, 9.157601356506348,
        6.385989189147949, 4.594970703125, 1.2000325918197632,
        3.1077473163604736, '51320_2015', 0],
       [3, 5.043741703033447, 3.5229599475860596, 7.4684038162231445,
        9.6045560836792, 3.8596458435058594, 9.537117004394531,
        6.133909225463867, 4.508759498596191, 0.9306520223617554,
        2.9496004581451416, '51320_2015', 0]], dtype=object)

In [ ]:
dsn = DeepSigNet()

In [ ]:
lass DeepSigNet(nn.Module):
    def __init__(
        self,
        input_channels,
        output_channels,
        sig_d,
        post_dim,
        hidden_dim,
        output_dim,
        dropout_rate,
        add_time=False,
        augmentation_tp="Conv1d",
        augmentation_layers=(),
    )